In [ ]:
import numpy as np
import torch
torch.set_default_dtype(torch.float64)

import h5py as h5

import tdg
import tdg.HMC as HMC

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
storage = 'ensemble-io.h5'

Let's study a small, hot example for computational simplicity.

In [ ]:
nx = 11
lattice = tdg.Lattice(nx)

ere = tdg.EffectiveRangeExpansion(torch.tensor([1.0]))

tuning = tdg.AnalyticTuning(ere, lattice)
print(f'{tuning.C[0]}')

In [ ]:
nt = 24
beta = torch.tensor(0.04)
mu = torch.tensor(3.125)
h  = torch.tensor([0,0,0], dtype=torch.float64)

S = tuning.Action(nt, beta, mu, h)

We could in principle use one Hamiltonian to do do the HMC Metropolis-Hastings accept/reject step and another to do the molecular dynamics integration.

Here we use the same Hamiltonian for both.

In [ ]:
H = HMC.Hamiltonian(S)

A = HMC.Autotuner(H, HMC.Omelyan, cfgs_per_estimate=20)
integrator, start = A.target(0.75, start='hot', starting_md_steps=20, progress=tqdm)

fig, ax = plt.subplots(2,1, figsize=(8,6))
A.plot_history(ax[0])
A.plot_models(ax[1])
ax[1].legend()
fig.tight_layout()

Wow!  We can do REALLY rough integration and still get high acceptance!

Let's start from the configuration generated by all the molecular dynamics integration during the autotuning!

In [ ]:
hmc = HMC.MarkovChain(H, integrator)
ensemble = tdg.ensemble.GrandCanonical(S).generate(100, hmc, start=start, progress=tqdm)

We can visualize an observable.

In [ ]:
ensemble.measure()

In [ ]:
import tdg.plot as visualize

In [ ]:
viz = visualize.History()
viz.plot(ensemble.N.real)
viz.plot(ensemble.N_bosonic.real)

Let's write it to storage and read it back in, but only read N observable.

In [ ]:
with h5.File(storage, 'w') as f:
    ensemble.to_h5(f.create_group('/example'))

The observables are 100 long

In [ ]:
with h5.File(storage, 'r') as f:
    for o in f['/example']:
        try:
            print(f'{o} ',f[f'/example/{o}'].shape)
        except:
            pass

In [ ]:
with h5.File(storage, 'r') as f:
    ENSEMBLE = tdg.ensemble.GrandCanonical.from_h5(f['/example'], observables=('N',))

In [ ]:
# To make the same figure we need to recompute N_bosonic...
viz = visualize.History()
viz.plot(ENSEMBLE.N.real, label='Read fermionic N')
viz.plot(ENSEMBLE.N_bosonic.real, label='Recomputed bosonic N')
viz.plot(ensemble.N_bosonic.real, label='Known bosonic N')

And now perhaps we wish to continue HMC!

In [ ]:
# To continue we don't need any observables, and we only need the last configuration
with h5.File(storage, 'r') as f:
    ENSEMBLE = tdg.ensemble.GrandCanonical.from_h5(f['/example'], selection=[-1,], observables=())
    print(f'The ENSEMBLE has length {len(ENSEMBLE)}')
    
INTEGRATOR = ENSEMBLE.generator
ACTION = ENSEMBLE.Action
LAST = ENSEMBLE.configurations[-1]
continuation = tdg.ensemble.GrandCanonical(ACTION).generate(100, INTEGRATOR, LAST, progress=tqdm)

In [ ]:
viz = visualize.History(2)
viz.plot(continuation.S.real, 0)
viz.plot(continuation.N.real, 1)
viz.plot(continuation.N_bosonic.real, 1)

Now we can measure and extend the results on disk...

In [ ]:
continuation.measure()

In [ ]:
with h5.File(storage, 'a') as f:
    continuation.extend_h5(f['/example'])

Now the extended quantities are 200 long:

In [ ]:
with h5.File(storage, 'r') as f:
    for o in f['/example']:
        try:
            print(f'{o} ',f[f'/example/{o}'].shape)
        except:
            pass

In [ ]:
with h5.File(storage, 'r') as f:
    everything = tdg.ensemble.GrandCanonical.from_h5(f['/example'], observables=('N', 'N_bosonic', 'S'))

viz = visualize.History(2)
viz.plot(everything.S.real, 0)
viz.plot(everything.N.real, 1)

viz.plot(ensemble.S.real, 0)
viz.plot(ensemble.N.real, 1)

viz.plot(continuation.S.real, 0, x=continuation.index+100)
viz.plot(continuation.N.real, 1, x=continuation.index+100)